# 8강) Dense Embedding 을 활용한 ODQA 시스템 만들기

### Requirements

In [1]:
# > /dev/null 2>&1 # execute command in silence
!pip install datasets==1.4.1 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.4.1 > /dev/null 2>&1
!pip install tqdm==4.41.1 > /dev/null 2>&1

In [2]:
import torch
import random
from pprint import pprint

## Dense Embedding 을 활용한 Open-domain Question Answering 시스템 만들기


5강에서 배운대로 dense embedding을 만드는 encoder 을 학습시키기

또는 학습된 encoder 파일을 가져와서 진행하기

In [ ]:
# 1. 미리 학습해둔 encoder file 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eMfMzv0gkcTSBQAxtQrFMP_pC5sEeQQq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1eMfMzv0gkcTSBQAxtQrFMP_pC5sEeQQq" -O dense_encoder.tar.gz && rm -rf /tmp/cookies.txt
# 2. .tar.gz file 압축해제 
!tar -xf ./dense_encoder.tar.gz

# 3. (Optional) 직접 학습 시킬수도 있음 (시간 오래걸림 주의)
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1U_5RI7je5p66RusSMrOMWJeJGEt9TK0i' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1U_5RI7je5p66RusSMrOMWJeJGEt9TK0i" -O train_encoder.py && rm -rf /tmp/cookies.txt
# 4. run train_encoder.py (will save the model in ~~)
# !python train_encoder.py

In [4]:
# google drive 에 올려둔 미리 학습해둔 인코더 불러오기
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer

class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output

model_checkpoint = "bert-base-multilingual-cased"
p_encoder = BertEncoder.from_pretrained(model_checkpoint).cuda()
q_encoder = BertEncoder.from_pretrained(model_checkpoint).cuda()
model_dict = torch.load("./dense_encoder/encoder.pth")
p_encoder.load_state_dict(model_dict['p_encoder'])
q_encoder.load_state_dict(model_dict['q_encoder'])

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias',

## Test

In [23]:
p_encoder = BertEncoder.from_pretrained(model_checkpoint).cuda()
p_encoder

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertEncoder(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [24]:
p_encoder.load_state_dict(model_dict['p_encoder'])
p_encoder

BertEncoder(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [22]:
q_encoder

BertEncoder(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [5]:
from datasets import load_dataset
dataset = load_dataset("squad_kor_v1")
# metric = load_metric('squad')

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


사전에 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [6]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [7]:
def get_relevant_doc(q_encoder, p_encoder, query, k=1):
    with torch.no_grad():
        p_encoder.eval()
        q_encoder.eval()

        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

        p_embs = []
        for p in valid_corpus:
            p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
            p_emb = p_encoder(**p).to('cpu').numpy()
            p_embs.append(p_emb)

    p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)
    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))

    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
    
    return dot_prod_scores.squeeze(), rank[:k]

In [34]:
random.seed(2020)
valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10] # context 들
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print("{} {} {}".format('*'*20, 'Ground Truth','*'*20))
print("[Search query]\n", query, "\n")
pprint(ground_truth, compact=True)

# valid_corpus

******************** Ground Truth ********************
[Search query]
 1960년 북한의 무장공비에게 살해당한 김영삼의 어머니 이름은? 

('제14대 대통령 선거에서 김영삼, 김대중의 출현을 못마땅하게 여기던 반공주의 세력과 군사 정권 세력은 야당 출신 인사들이 북한과 '
 '거래한다는 투의 의혹을 제기했다. 그러자 김영삼은 1960년 자신의 어머니 박부련이 무장공비에게 살해당한 점을 들어 위기를 모면하였다. '
 '한편 민자당내 군사정권 출신 세력은 김영삼의 지도권에 반발하였으나, 대구 경북출신의 군사정권 인사 정호용이 그에 대한 지지를 촉구하여 '
 '일시적으로 봉합시키기도 했다. 그러나 정호용의 ‘우리는 남이가, 같은 경상도이니 지지하자’ 는 만류에 의해 군사 정권 세력과 반공주의 '
 '세력은 김영삼 지지층과 김종필 지지로 나뉘었고, 그는 구 민주당계 인사와 경상남도, 부산 지역의 표심을 장악하여 대통령에 무난히 당선될 '
 '수 있었다.')


In [9]:
_, doc_id = get_relevant_doc(p_encoder, q_encoder, query, k=1)

""" 상위 1개 문서를 추출했을 때 결과 확인 """
print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id}")
print(valid_corpus[doc_id.item()])
# print(answer)

******************** Result ********************
[Search query]
 1960년 북한의 무장공비에게 살해당한 김영삼의 어머니 이름은? 

[Relevant Doc ID(Top 1 passage)]: tensor([10])
제14대 대통령 선거에서 김영삼, 김대중의 출현을 못마땅하게 여기던 반공주의 세력과 군사 정권 세력은 야당 출신 인사들이 북한과 거래한다는 투의 의혹을 제기했다. 그러자 김영삼은 1960년 자신의 어머니 박부련이 무장공비에게 살해당한 점을 들어 위기를 모면하였다. 한편 민자당내 군사정권 출신 세력은 김영삼의 지도권에 반발하였으나, 대구 경북출신의 군사정권 인사 정호용이 그에 대한 지지를 촉구하여 일시적으로 봉합시키기도 했다. 그러나 정호용의 ‘우리는 남이가, 같은 경상도이니 지지하자’ 는 만류에 의해 군사 정권 세력과 반공주의 세력은 김영삼 지지층과 김종필 지지로 나뉘었고, 그는 구 민주당계 인사와 경상남도, 부산 지역의 표심을 장악하여 대통령에 무난히 당선될 수 있었다.


In [10]:
""" 상위 5개를 추출하여 점수 확인 """
dot_prod_scores, rank = get_relevant_doc(p_encoder, q_encoder, query, k=5)

for i in range(5):
    print(rank[i])
    print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
    print(valid_corpus[rank[i]])

tensor(10)
Top-1 passage with score 187.5587
제14대 대통령 선거에서 김영삼, 김대중의 출현을 못마땅하게 여기던 반공주의 세력과 군사 정권 세력은 야당 출신 인사들이 북한과 거래한다는 투의 의혹을 제기했다. 그러자 김영삼은 1960년 자신의 어머니 박부련이 무장공비에게 살해당한 점을 들어 위기를 모면하였다. 한편 민자당내 군사정권 출신 세력은 김영삼의 지도권에 반발하였으나, 대구 경북출신의 군사정권 인사 정호용이 그에 대한 지지를 촉구하여 일시적으로 봉합시키기도 했다. 그러나 정호용의 ‘우리는 남이가, 같은 경상도이니 지지하자’ 는 만류에 의해 군사 정권 세력과 반공주의 세력은 김영삼 지지층과 김종필 지지로 나뉘었고, 그는 구 민주당계 인사와 경상남도, 부산 지역의 표심을 장악하여 대통령에 무난히 당선될 수 있었다.
tensor(4)
Top-2 passage with score 179.1218
연이어 9월 16일에 개봉한 영화 《사도》에서는 뒤주에서 생을 마감하는 ‘사도세자’를 연기했다. 촬영 도중 돌에 머리를 찧어 부상을 입기도 했다. 유아인은 《사도》에 출연한 것에 대해 “영광스럽다. 젊은 배우가 좋은 배역을 만난다는 게 쉽지 않다. 어쩌다보니 행운 같은 순간들이 찾아와서 이런 깊은 감정을 연기할 수 있게 돼 행복하다”고 말했다. 게다가 사도세자 역할은 배우로서 그려왔던 불안한 청춘과 반항아 등의 집약체 같은 인물이기 때문에 "12년간 준비해왔어요"라고 말할 수 있을 정도로 "내가 조금씩 추구해왔고 걸어왔던 길의 정점이 된 것 같다"라고 설명한다. 이 영화에서 유아인은 숙명으로 인해 광인으로 변하는 인물의 감정을 전달했으며, 함께 한 배우 송강호와 이준익 감독으로부터 테크닉을 경계하고 정직하게 본성을 표출하며 연기했다는 말을 들었다.
tensor(9)
Top-3 passage with score 178.9075
영국에서는 쇼와 천황이 죽은 직후 언론들을 중심으로 “쇼와 천황은 악마의 화신”이라는 비판이 쏟아졌다. 영국의 

## 훈련된 MRC 모델 가져오기


In [11]:
import torch
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

In [12]:
model_name = 'sangrimlee/bert-base-multilingual-cased-korquad'
mrc_model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [13]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)

In [14]:
mrc_model = mrc_model.eval()

In [38]:
def get_answer_from_context(context, question, model, tokenizer):
    encoded_dict = tokenizer.encode_plus(  
        question,
        context,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    non_padded_ids = encoded_dict["input_ids"][: encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
    full_text = tokenizer.decode(non_padded_ids)
 
    inputs = {
    'input_ids': torch.tensor([encoded_dict['input_ids']], dtype=torch.long),
    'attention_mask': torch.tensor([encoded_dict['attention_mask']], dtype=torch.long),
    'token_type_ids': torch.tensor([encoded_dict['token_type_ids']], dtype=torch.long)
    }

    outputs = model(**inputs)
    start, end = torch.max(outputs.start_logits, axis=1).indices.item(), torch.max(outputs.end_logits, axis=1).indices.item()

    if start == 0 and end == 0:
        answer = "This is not answerable"
    else:
        answer = tokenizer.decode(encoded_dict['input_ids'][start:end+1])
    return answer

In [39]:
context = valid_corpus[doc_id.item()]
answer = get_answer_from_context(context, query, mrc_model, tokenizer)
print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
pprint(valid_corpus[doc_id.item()], compact=True)
print(f"[Answer Prediction from the model]: {answer}")

******************** Result ********************
[Search query]
 1960년 북한의 무장공비에게 살해당한 김영삼의 어머니 이름은? 

[Relevant Doc ID(Top 1 passage)]: 10
('제14대 대통령 선거에서 김영삼, 김대중의 출현을 못마땅하게 여기던 반공주의 세력과 군사 정권 세력은 야당 출신 인사들이 북한과 '
 '거래한다는 투의 의혹을 제기했다. 그러자 김영삼은 1960년 자신의 어머니 박부련이 무장공비에게 살해당한 점을 들어 위기를 모면하였다. '
 '한편 민자당내 군사정권 출신 세력은 김영삼의 지도권에 반발하였으나, 대구 경북출신의 군사정권 인사 정호용이 그에 대한 지지를 촉구하여 '
 '일시적으로 봉합시키기도 했다. 그러나 정호용의 ‘우리는 남이가, 같은 경상도이니 지지하자’ 는 만류에 의해 군사 정권 세력과 반공주의 '
 '세력은 김영삼 지지층과 김종필 지지로 나뉘었고, 그는 구 민주당계 인사와 경상남도, 부산 지역의 표심을 장악하여 대통령에 무난히 당선될 '
 '수 있었다.')
[Answer Prediction from the model]: 박부련


## 통합해서 ODQA 시스템 구축

In [17]:
def open_domain_qa(query, corpus, p_encoder, q_encoder, mrc_model, tokenizer, k=1):
    # 1. Retrieve k relevant docs by usign sparse matrix
    _, doc_id = get_relevant_doc(p_encoder, q_encoder, query, k=1)
    context = corpus[doc_id.item()]

    # 2. Predict answer from given doc by using MRC model
    answer = get_answer_from_context(context, query, mrc_model, tokenizer)
    print("{} {} {}".format('*'*20, 'Result','*'*20))
    print("[Search query]\n", query, "\n")
    print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
    pprint(corpus[doc_id.item()], compact=True)
    print(f"[Answer Prediction from the model]: {answer}")

In [40]:
query = input("Enter any question: ") # "대한민국의 대통령은 누구인가?"
open_domain_qa(query, valid_corpus, p_encoder, q_encoder, mrc_model, tokenizer, k=1)

******************** Result ********************
[Search query]
 "대한민국의 대통령은 누구인가? 

[Relevant Doc ID(Top 1 passage)]: 10
('제14대 대통령 선거에서 김영삼, 김대중의 출현을 못마땅하게 여기던 반공주의 세력과 군사 정권 세력은 야당 출신 인사들이 북한과 '
 '거래한다는 투의 의혹을 제기했다. 그러자 김영삼은 1960년 자신의 어머니 박부련이 무장공비에게 살해당한 점을 들어 위기를 모면하였다. '
 '한편 민자당내 군사정권 출신 세력은 김영삼의 지도권에 반발하였으나, 대구 경북출신의 군사정권 인사 정호용이 그에 대한 지지를 촉구하여 '
 '일시적으로 봉합시키기도 했다. 그러나 정호용의 ‘우리는 남이가, 같은 경상도이니 지지하자’ 는 만류에 의해 군사 정권 세력과 반공주의 '
 '세력은 김영삼 지지층과 김종필 지지로 나뉘었고, 그는 구 민주당계 인사와 경상남도, 부산 지역의 표심을 장악하여 대통령에 무난히 당선될 '
 '수 있었다.')
[Answer Prediction from the model]: 정호용이
